# Find Teleconnections (precursor regions) via correlation maps 

In [ ]:
import os, inspect
main_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) # script directory
print(main_dir)
%load_ext autoreload
%autoreload 2
from RGCPD import RGCPD
from RGCPD import BivariateMI
import class_BivariateMI, functions_pp
import numpy as np
from IPython.display import Image

## RGCPD docstring
Class to study teleconnections of a Response Variable* of interest.

    Methods to extract teleconnections/precursors:
        - BivariateMI (supporting (partial) correlation maps)
        - EOF analysis

    BivariateMI (MI = Mutual Information) is class which allows for a
    statistical test in the form:
    MI(lon,lat) = for gc in map: func(x(t), y(t)),
    where map is a (time,lon,lat) map and gc stands for each gridcell/coordinate
    in that map. The y timeseries is always the same 1-dimensional timeseries of
    interest (i.e. the Response Variable). At this point, only supports the
    correlation analysis. Once the significance is attributed, it is stored
    in the MI map. Precursor regions are found by clustering the
    significantly (correlating) gridcells (+ and - regions are separated)
    and extract their spatial mean (or spatial covariance) timeseries.

    *Sometimes Response Variable is also called Target Variable.

    Parameters
    ----------
    list_of_name_path : list, optional
        list of (name, path) tuples defining the input data.

        Convention: first entry should be (name, path) of target variable (TV).
        e.g. list_of_name_path = [('TVname', 'TVpath'), ('prec_name', 'prec_path')]

        TVpath input data supports .nc/.h5 or .csv file format.
        if using output of the clustering:
            'TVname' should refer the name
            you have given the timeseries on the dimesion 'cluster', i.e.
            xrTV.sel(cluster=TVname)
        elif using .h5 the index should contain a datetime axis.
        elif using .csv the first columns should be [year, month, day, value]


        prec_path input data supports only .nc
        'prec_name' is a string/key that can be chosen freely, does not have
        to refer to the variable in the .nc file.
        Each prec_path .nc file should contain only a single variable
        of format (time, lat, lon).
    list_for_EOFS : list, optional
        list of EOF classes, see docs EOF?
    list_import_ts : list, optional
        Load in precursor 1-d timeseries in format:
        [(name1, path_to_h5_file1), [(name2, path_to_h5_file2)]]
        precursor_ts can handle the RGCPD cross-validation format.
    start_end_TVdate : tuple, optional
        tuple of start- and enddate for target variable in
        format ('mm-dd', 'mm-dd').
    tfreq : int, optional
        The default is 10, if using time_mean_periods, tfreq should be None.
    start_end_date : tuple, optional
        tuple of start- and enddate for data to load in
        format ('mm-dd', 'mm-dd'). default is ('01-01' - '12-31')
    start_end_year : tuple, optional
        default is to load all years
    path_outmain : [str, bool], optional
        Root folder for output. If None, default is your
        '/users/{username}/Download' path.
    append_pathsub: str, optional
        The first subfolder will be created below path_outmain, to store
        output data & figures. The append_pathsub1 argument allows you to
        manually add some hash or string refering to some experiment.
    save : bool, optional
        If you want to save figures, data and text output automatically.
    verbosity : int, optional
        Regulate the amount of feedback given by the code.
        The default is 1.

    Returns
    -------
    initialization of the RGCPD class

In [ ]:
path_test = os.path.join(main_dir, 'data') # path of test data
# define input by filling list_of_name_path: 
# format list_of_name_path = [('TVname', 'TVpath'), ('prec_name', 'prec_path')]
list_of_name_path = [(3, os.path.join(path_test, 'tf5_nc5_dendo_80d77.nc')),
                    ('sst', os.path.join(path_test,'sst_daily_1979-2018_5deg_Pacific_175_240E_25_50N.nc'))]

# define analysis:
list_for_MI = [BivariateMI(name='sst', func=class_BivariateMI.corr_map, 
                           alpha=.01, FDR_control=True, lags=np.array([1]), 
                           distance_eps=700, min_area_in_degrees2=5)]

In [ ]:
?BivariateMI

# Time-handling subseasonal mode (DJF)

In [ ]:
rg = RGCPD(list_of_name_path=list_of_name_path,
           list_for_MI=list_for_MI,
           tfreq=10, # <- subseasonal forecasting mode, all data will be aggregated to 10-day means
           start_end_TVdate=('02-28', '12-01'), # <- defining DJF target period 
           start_end_date=('02-28', '07-01'), # <- Loading subset of data for computational efficiency
           path_outmain=os.path.join(main_dir,'data'))

In [ ]:
Image(filename = "./data/subseasonal_mode_DJF.png", width = 400, height = 200)

# Time-handling subseasonal mode (JJA)

In [ ]:
rg = RGCPD(list_of_name_path=list_of_name_path,
           list_for_MI=list_for_MI,
           tfreq=10, # <- subseasonal forecasting mode, all data will be aggregated to 10-day means
           start_end_TVdate=('06-01', '08-31'), # <- defining target period 
           path_outmain=os.path.join(main_dir,'data'))

In [ ]:
Image(filename = "./data/subseasonal_mode_JJA.png", width = 400, height = 200)

Note, if loading in monthly data, the exact same reasoning is followed. Where tfreq now stands for n-month means instead of n-day means.

In [ ]:
# if TVpath contains the xr.DataArray that is clustering beforehand, we can have a look at the spatial regions.
rg.plot_df_clust()

In [ ]:
rg.pp_precursors(detrend=True, anomaly=True, selbox=None)

Option for pp_precursors to vary pre-processing of datasets by given a list as an argument:
An example: detrend=[True, {'sm1':False, 'sm2':False}]. This input means that default argument is True, except the variables sm1 and sm2 got a different argument. The precursor names sm1 and sm2 should refer to the names given in list_of_name_path.

### Post-processing Target Variable

In [ ]:
rg.pp_TV?

In [ ]:
rg.pp_TV(detrend=True, anomaly=True)

In [ ]:
rg.fulltso

In [ ]:
rg.traintest?

In [ ]:
rg.traintest(method='random_5')
# rg._get_testyrs()[0] # see test years

In [ ]:
rg.calc_corr_maps() 

In [ ]:
rg.plot_maps_corr()

In [ ]:
rg.cluster_list_MI()

In [ ]:
rg.quick_view_labels(mean=False) 

In [ ]:
rg.get_ts_prec()

From the correlation (BivariateMI) map, the timeseries are named according the format:
{lag}..{region label}..{precursor name}, where precursor name is defined in list_of_name_path. 

In [ ]:
rg.df_data

## Train-test cross-validation and Response Variable mask handling

- The different train-test fold organized in the first level index.
- Each fold contains precursor timeseries that were extracted from only training data.
- The Training data is specified by the TrainIsTrue mask.
- The Response Variable mask defined the dates to predict 
- The RV_mask the anker for lag shifting the precursors.

The last two columns should be always present. 
- The RV_mask is defined by the start_end_TVdate.
- The TrainIsTrue is defined by the method for cross-validation.


In [ ]:
# Example lag shifting
import func_models as fc_utils
fc_utils.apply_shift_lag(rg.df_splits.loc[0].copy(), lag_i=1)

## Causal Discovery using Tigramite

In [ ]:
rg.PCMCI_df_data(tigr_function_call='run_pcmci',
                 kwrgs_tigr={'tau_min': 0,
                             'tau_max': 1,
                             'pc_alpha': 0.05,
                             'max_conds_dim': 2,
                             'max_combinations': 2,
                             'max_conds_py': 2,
                             'max_conds_px': 2})

In [ ]:
rg.PCMCI_get_links?

In [ ]:
rg.PCMCI_get_links(var=rg.TV.name, alpha_level=.05)
rg.df_links

In [ ]:
# on the left you see all the extraction precursor regions, on the right you see the regions which were found Conditionally Dependent (~Causal)
rg.plot_maps_sum()

In [ ]:
# Deprecated: rg.df_ParCorr_sum 

## Functionality for Causal Inference
(Define your own Conditional Independence tests)

In [ ]:
import wrapper_PCMCI
corr, pvals = wrapper_PCMCI.df_data_Parcorr(rg.df_data, 
                                            target='3ts',
                                            keys=['1..2..sst'],
                                            z_keys=['1..1..sst'],
                                            z_lag=1)
pvals

In [ ]:
df_trans = rg.transform_df_data()

df_z_removed = wrapper_PCMCI.df_data_remove_z(df_trans,
                                              keys=['3ts'],
                                              z=['1..2..sst'],
                                              plot=True)

# Forecasting pipeline 1

In [ ]:
import func_models as fc_utils
from stat_models_cont import ScikitModel
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegressionCV


# choose type prediciton (continuous or probabilistic) by making comment #
prediction = 'continuous'   
# prediction = 'events' ; q = .66 # quantile threshold for event definition

if prediction == 'continuous':
    model = ScikitModel(Ridge, verbosity=0)
    # You can also tune parameters by passing a list of values. Then GridSearchCV from sklearn will 
    # find the set of parameters that give the best mean score on all kfold test sets. 
    # below we pass a list of alpha's to tune the regularization.
    alphas = list(np.concatenate([[1E-20],np.logspace(-5,0, 6), np.logspace(.01, 2.5, num=25)]))                       
    kwrgs_model = {'scoringCV':'neg_mean_absolute_error',
                   'kfold':5,
                   'alpha':alphas} # large a, strong regul.
elif prediction == 'events':
    model = ScikitModel(LogisticRegressionCV, verbosity=0)
    kwrgs_model = {'kfold':5,
                   'scoring':'neg_brier_score'}

    

target_ts = rg.TV.RV_ts ; 
target_ts = (target_ts - target_ts.mean()) / target_ts.std()
if prediction == 'events':
    if q >= 0.5:
        target_ts = (target_ts > target_ts.quantile(q)).astype(int)
    elif q < .5:
        target_ts = (target_ts < target_ts.quantile(q)).astype(int)
    BSS = fc_utils.ErrorSkillScore(constant_bench=float(target_ts.mean())).BSS
    score_func_list = [BSS, fc_utils.metrics.roc_auc_score]
    
elif prediction == 'continuous':
    RMSE_SS = fc_utils.ErrorSkillScore(constant_bench=float(target_ts.mean())).RMSE
    MAE_SS = fc_utils.ErrorSkillScore(constant_bench=float(target_ts.mean())).MAE
    score_func_list = [RMSE_SS, fc_utils.corrcoef, MAE_SS]
        
    
out = rg.fit_df_data_ridge(target=target_ts,
                            keys=None,
                            fcmodel=model,
                            kwrgs_model=kwrgs_model,
                            transformer=False,
                            tau_min=1, tau_max=3)
predict, weights, model_lags = out

df_train_m, df_test_s_m, df_test_m, df_boot = fc_utils.get_scores(predict,
                                                                 rg.df_data.iloc[:,-2:],
                                                                 score_func_list,
                                                                 n_boot = 100,
                                                                 score_per_test=False,
                                                                 blocksize=1,
                                                                 rng_seed=1)
lag = 1
if prediction == 'events':
    print(model.scikitmodel.__name__, '\n', f'Test score at lag {lag}\n',
          'BSS {:.2f}\n'.format(df_test_m.loc[0].loc[lag].loc['BSS']),
          'AUC {:.2f}'.format(df_test_m.loc[0].loc[lag].loc['roc_auc_score']),
          '\nTrain score\n',
          'BSS {:.2f}\n'.format(df_train_m.mean(0).loc[lag]['BSS']),
          'AUC {:.2f}'.format(df_train_m.mean(0).loc[lag]['roc_auc_score']))
elif prediction == 'continuous':
    print(model.scikitmodel.__name__, '\n', 'Test score\n',
              'RMSE {:.2f}\n'.format(df_test_m.loc[0][lag]['RMSE']),
              'MAE {:.2f}\n'.format(df_test_m.loc[0][lag]['MAE']),
              'corrcoef {:.2f}'.format(df_test_m.loc[0][lag]['corrcoef']),
              '\nTrain score\n',
              'RMSE {:.2f}\n'.format(df_train_m.mean(0).loc[lag]['RMSE']),
              'MAE {:.2f}\n'.format(df_train_m.mean(0).loc[lag]['MAE']),
              'corrcoef {:.2f}'.format(df_train_m.mean(0).loc[lag]['corrcoef']))

In [ ]:
df_test_m.loc[0].plot.bar(rot=0, color=['blue', 'green', 'purple'], figsize=(10,4))

In [ ]:
from stat_models import plot_importances
plot_importances(models_splits_lags=model_lags, lag=1)

### Match the retrieved timeseries of BivariateMI to the lag of forecasting 
Quick summary of code:

In [ ]:
rg.list_for_MI[0].lags = np.array([0,1,2]) # update lags correlation map
rg.calc_corr_maps()
rg.cluster_list_MI()
rg.get_ts_prec()
rg.df_data

Above, we now have a timeseries of the correlation maps lag 0 up to lag 2. Below, we can match these timeseries to lag-of-forecast by setting 'match_lag_region_to_lag_fc' to True.

In [ ]:
rg.fit_df_data_ridge(match_lag_region_to_lag_fc=True,
                    tau_min=0, tau_max=2)
# Predicitons for lag 0, 1 and 2 using precursors extraction from correlation maps at lag 0, 1, 2, respecitvely.

## The arguments for using this package 
- Reproducible
- Flexible
- Time efficient

### An example of flexibility

In [ ]:
# set up your pipeline
import func_models as fc_utils
def pipeline(tfreq, start_end_TVdate):
    rg = RGCPD(tfreq=tfreq, start_end_TVdate=start_end_TVdate)
    rg.list_for_MI = [BivariateMI(name='sst_test', distance_eps=700)]
    rg.pp_precursors() ; rg.pp_TV()
    rg.traintest('random_5')
    rg.calc_corr_maps() ; rg.cluster_list_MI()
    rg.get_ts_prec()
    prediction = rg.fit_df_data_ridge()[0]
    out = fc_utils.get_scores(prediction,
                              rg.df_data.iloc[:,-2:],
                              None,
                              n_boot = 0,
                              score_per_test=False,
                              blocksize=1)
    df_train_m, df_test_s_m, df_test_m, df_boot = out
    print(df_test_m.loc[0])  

Now lets choose a common timescale and target period for seasonal forecasts. 

In [ ]:
pipeline(7, ('06-01','08-31')) 

With the code handling all the anoying data-handling stuff, one can easily test different settings. 

In [ ]:
pipeline(15, ('07-01','08-31'))

Note a 30% increase in skill (MSE). The commonly used target window and timescale had no good predictability 
(likely has to due with dependence of temperature on end-of-summer soil moisture). 
\
\
When one codes a pipeline for 1 goal (7 day-mean forecast, JJA summer). He/she is restricted by the investment when testing other plausible hypothesis. With this Python package, users are freed from the coding time-investment and can spend more time on testing different hypthesis very efficiently.

A non-extensive list of options to test:
- Different timescales
- Different target window(s)
- Different precursors
- Different precursor extraction methods ((partial) correlation, EOF, ...)
- Different statistical model feature selection methods
- Different statistical models / tuning methods
- Different train-test splits
- Different definitions for the target

## (old) Forecasting pipeline 2 
Used for MWR paper https://doi.org/10.1175/MWR-D-19-0409.1

There is some multiprocessing based on Python's standard 'concurrent futures' module. This only works when run script is run in one go. Will not work another time. Has to do with the running the code as the __main__ file or something.. (don't know the details). 

In [ ]:
rg.store_df_PCMCI() # storing timeseries in .h5 format
path_df_data = rg.path_df_data

Now we load in the data, including info on the causal links. 

In [ ]:
from class_fc import fcev
import valid_plots as dfplots

In [ ]:
fc = fcev(path_data=path_df_data, n_cpu=1, causal=True)
fc.df_data

In [ ]:
fc.get_TV(kwrgs_events=None)

Define statmodel:

In [ ]:
fc.fit_models?


In [ ]:
fc.fit_models(lead_max=35)

In [ ]:
dict_experiments = {}       
fc.perform_validation(n_boot=100, blocksize='auto', 
                              threshold_pred=(1.5, 'times_clim'))
dict_experiments['test'] = fc.dict_sum

In [ ]:
working_folder, filename = fc._print_sett(list_of_fc=[fc])
store=True
dict_all = dfplots.merge_valid_info([fc], store=store)
if store:
    dict_merge_all = functions_pp.load_hdf5(filename)

In [ ]:

kwrgs = {'wspace':0.25, 'col_wrap':3} #, 'threshold_bin':fc.threshold_pred}
met = ['AUC-ROC', 'AUC-PR', 'BSS', 'Rel. Curve', 'Precision', 'Accuracy']
expers = list(dict_experiments.keys())
line_dim = 'model'


fig = dfplots.valid_figures(dict_merge_all, #expers=expers, #models=models,
                          line_dim=line_dim, 
                          group_line_by=None,  
                          met=met, **kwrgs)